# Ejemplo de modelo en Pringles

### 1. Discover del codigo de atomicos

Empezamos con los modelos construidos en user_models/ . En este caso tenemos:

In [ ]:
!find user_models/ -name "*.h"

In [ ]:
from pringles.simulator import Simulator

In [ ]:
simulator = Simulator("bin/", "user_models/")
r = simulator.get_registry()

Todas estas clases que heredan de Atomic fueron creadas automaticamente a partir del codigo fuente

In [ ]:
r.discovered_atomics

Y cada una puede ser accedida asi:

In [ ]:
r.Paranoia

### 2. Construir el modelo usando la interfaz en python

In [ ]:
from pringles.models import Coupled

In [ ]:
def crear_interacciones_poblacion(mean=2, std=1, threshold_NV=30, threshold_V=80):
    # Los atomicos ya tienen los puertos definidos a partir de leer la metadata del codigo
    un_foco = r.Foco("foco", mean=mean, std=std)
    un_contagio = r.Contagio("contagio", threshold_NV=threshold_NV, threshold_V=threshold_V)

    interacciones_poblacion = (Coupled("interacciones_poblacion", [un_foco, un_contagio])
        .add_inport("in_port")
        .add_outport("out_port")
        .add_coupling("in_port", un_foco.get_port("in"))
        .add_coupling(un_foco.get_port("out"), un_contagio.get_port("in"))
        .add_coupling(un_contagio.get_port("out"), "out_port"))
    return interacciones_poblacion

In [ ]:
interacciones_pob = crear_interacciones_poblacion()
interacciones_pob

In [ ]:
def crear_sist_vacunacion(p_inf_NV=0.3, p_inf_V=0.8, alpha=2, cant_vacunas_por_dia_activo=10000, prop_infectados_inicio=0.1, prop_infectados_cierre=0.025):
    un_sist_prevencion = r.SistPrevencion("sist_prevencion", prop_infectados_inicio=prop_infectados_inicio, prop_infectados_cierre=prop_infectados_cierre)
    un_vacunatorio = r.Vacunatorio("vacunatorio", cant_vacunas_por_dia_activo=cant_vacunas_por_dia_activo)
    un_paranoia = r.Paranoia("paranoia", p_inf_NV=p_inf_NV, p_inf_V=p_inf_V, alpha=alpha)
    
    sist_vacunacion = (Coupled("sist_vacunacion", [un_sist_prevencion, un_vacunatorio, un_paranoia])
        .add_inport('in_port')
        .add_outport('out_port')
        .add_coupling('in_port', un_paranoia.get_port("in"))
        .add_coupling('in_port', un_sist_prevencion.get_port("in"))
        .add_coupling(un_paranoia.get_port("out"), un_vacunatorio.get_port("in_pacientes"))
        .add_coupling(un_sist_prevencion.get_port("out"), un_vacunatorio.get_port("in_activo"))
        .add_coupling(un_vacunatorio.get_port("out"), "out_port")
    )
    return sist_vacunacion

In [ ]:
sist_vac = crear_sist_vacunacion()
sist_vac

In [ ]:
def crear_top(susceptibles_nv=547250, susceptibles_v=447750,
              infectados_nv=2750, infectados_v=2250,
              recuperados_nv=0, recuperados_v=0,
              vacunados_nv=0, vacunados_v=0,
              proba_sana_nv=0.2, proba_sana_v=0.1):
    pobl = r.Poblacion("poblacion",
            susceptibles_nv=susceptibles_nv,
            susceptibles_v=susceptibles_v,
            infectados_nv=infectados_nv,
            infectados_v=infectados_v,
            recuperados_nv=recuperados_nv,
            recuperados_v=recuperados_v,
            vacunados_nv=vacunados_nv,
            vacunados_v=vacunados_v,
            proba_sana_nv=proba_sana_nv,
            proba_sana_v=proba_sana_v
    )
    sist_vac = crear_sist_vacunacion()
    interacciones_poblacion = crear_interacciones_poblacion()
    
    top = (Coupled("top", [pobl, sist_vac, interacciones_poblacion])
        .add_outport('out_port')
        .add_coupling(pobl.get_port('out'), sist_vac.get_port('in_port'))
        .add_coupling(pobl.get_port('out'), interacciones_poblacion.get_port('in_port'))
        .add_coupling(pobl.get_port('out'), 'out_port')
        .add_coupling(sist_vac.get_port('out_port'), pobl.get_port("in_deltavac"))
        .add_coupling(interacciones_poblacion.get_port('out_port'), pobl.get_port("in_deltainf"))
    )
    return top

In [ ]:
top = crear_top()
top

### 3. Correr simulation

In [ ]:
print(type(simulator))  # Instanciado arriba
print(type(top)) # instanciado arriba

In [ ]:
from pringles.utils import VirtualTime
from pringles.simulator import Simulation

In [ ]:
simulation = Simulation(top_model=top,
                        duration=VirtualTime.parse('00:00:30:000:0'))
result = simulator.run_simulation(simulation)

### 4. Analizar resultados

In [ ]:
result.successful()

In [ ]:
result.logs_dfs['ParallelRoot'].head()

In [ ]:
print(type(result.logs_dfs['ParallelRoot']['time'][0]))
print(type(result.logs_dfs['ParallelRoot']['value'][0]))

In [ ]:
import matplotlib.pyplot as plt
from pringles.utils import vtime_decorate
fig, ax = plt.subplots()
vtime_decorate(ax)
result.plot_port('top', 'out', axes=ax, index=0)
result.plot_port('top', 'out', axes=ax, index=1)
result.plot_port('top', 'out', axes=ax, index=2)
result.plot_port('top', 'out', axes=ax, index=3)